In [ ]:
# Instalación de PySpark
!pip install pyspark

# Importar las librerías necesarias
from pyspark.sql import SparkSession

In [ ]:
# Crear sesión de Spark
spark = SparkSession.builder.appName("Actividad PySpark").getOrCreate()

# Lista de datos de transacciones
transacciones = [
    ("Chile", "Electrónica", 1200),
    ("Perú", "Ropa", 800),
    ("México", "Electrónica", 1500),
    ("Chile", "Alimentos", 700),
    ("Perú", "Electrónica", 1000),
    ("México", "Ropa", 600)
]

In [ ]:
# PARTE 1: Crear y Manipular un RDD

# Crear un RDD a partir de la lista transacciones usando parallelize()
rdd = spark.sparkContext.parallelize(transacciones)

# Aplicar un filter() para seleccionar solo transacciones del país "Chile"
rdd_chile = rdd.filter(lambda x: x[0] == "Chile")

# Convertir el RDD filtrado en mayúsculas utilizando map()
rdd_chile_mayusculas = rdd_chile.map(lambda x: (x[0].upper(), x[1].upper(), x[2]))

print("Transacciones de Chile en mayúsculas:")
print(rdd_chile_mayusculas.collect())

Transacciones de Chile en mayúsculas:
[('CHILE', 'ELECTRÓNICA', 1200), ('CHILE', 'ALIMENTOS', 700)]


In [ ]:
# PARTE 2: Crear y consultar un DataFrame

# Convertir la lista transacciones en un DataFrame con columnas: País, Categoría, Monto
df = spark.createDataFrame(transacciones, ["Pais", "Categoria", "Monto"])

# Filtrar las transacciones donde el monto sea mayor a 1000
df_filtrado = df.filter(df["Monto"] > 1000)
print("Transacciones con monto mayor a 1000:")
df_filtrado.show()

# Consultar las ventas totales por país utilizando Spark SQL
df.createOrReplaceTempView("ventas_temp")
resultado_pais = spark.sql("SELECT Pais, SUM(Monto) AS Total FROM ventas_temp GROUP BY Pais")
print("Ventas totales por país:")
resultado_pais.show()

Transacciones con monto mayor a 1000:
+------+-----------+-----+
|  Pais|  Categoria|Monto|
+------+-----------+-----+
| Chile|Electrónica| 1200|
|México|Electrónica| 1500|
+------+-----------+-----+

Ventas totales por país:
+------+-----+
|  Pais|Total|
+------+-----+
| Chile| 1900|
|  Perú| 1800|
|México| 2100|
+------+-----+



In [ ]:
# PARTE 3: Consultas SQL sobre el DataFrame

# Registrar el DataFrame como tabla temporal
df.createOrReplaceTempView("ventas")

# Ejecutar una consulta SQL para obtener el total de ventas por categoría
resultado_categoria = spark.sql("SELECT Categoria, SUM(Monto) AS Total FROM ventas GROUP BY Categoria")

# Mostrar los resultados finales
print("Total de ventas por categoría:")
resultado_categoria.show()

Total de ventas por categoría:
+-----------+-----+
|  Categoria|Total|
+-----------+-----+
|       Ropa| 1400|
|Electrónica| 3700|
|  Alimentos|  700|
+-----------+-----+



## PARTE 4: Evaluación de la Actividad

### 1. Creación de RDD y filtrado correcto ✅
- **RDD original**: Creado exitosamente con 6 transacciones usando `parallelize()`
- **Filtrado**: RDD filtrado de Chile contiene 2 transacciones
- **Transformación**: Conversión a mayúsculas aplicada correctamente con `map()`
- **Resultado**: `[('CHILE', 'ELECTRÓNICA', 1200), ('CHILE', 'ALIMENTOS', 700)]`

### 2. Conversión a DataFrame y ejecución de consultas SQL ✅
- **DataFrame**: Creado con 6 filas y 3 columnas (Pais, Categoria, Monto)
- **Filtrado**: 3 transacciones superan los 1000 pesos
- **Consultas SQL**: Ejecutadas exitosamente para totales por país y categoría
- **Tabla temporal**: Registrada correctamente como 'ventas'

### 3. Interpretación correcta de los resultados obtenidos ✅

#### Análisis por País:
- **Chile**: 2 transacciones - Electrónica (1200) y Alimentos (700) = Total 1900
- **México**: 2 transacciones - Electrónica (1500) y Ropa (600) = Total 2100
- **Perú**: 2 transacciones - Ropa (800) y Electrónica (1000) = Total 1800

#### Análisis por Categoría:
- **Electrónica**: 3700 (Chile 1200 + México 1500 + Perú 1000)
- **Ropa**: 1400 (Perú 800 + México 600)
- **Alimentos**: 700 (Chile 700)

#### Conclusiones:
- ✅ **México** es el país con mayores ventas totales (2100)
- ✅ **Electrónica** es la categoría con mayores ventas (3700)
- ✅ Solo **3 transacciones** superan los 1000: Chile-Electrónica, México-Electrónica, Perú-Electrónica

In [ ]:
# Finalizar Spark
spark.stop()
print("Sesión de Spark finalizada.")